# СРС №3. Обслуживание простых деревьев. Перечисление путей
## Задание.
1. Реализовать структуру таблиц, реализовывающую структуру для хранения деревьев «Перечисление путей» и позволяющую хранить данные выбранной тематики. Тематика может быть выбрана из списка предложенных (см. Предлагаемые темы) или предложена студентом.
2. Наполнить данными реализованную структуру. Наполнение и генерация данных должны производиться с программным кодом. Таблица с деревом должна содержать не менее 150 строк, каждый узел, не являющийся листом должен иметь от 2 до 5 потомков, общая глубина дерева должна быть не менее 10.
3. Реализовать следующие операции в виде функций программной бибилотеки с текстовым или графическим интерфейсом:
- добавление листа;
- удаление листа;
- удаление поддерева;
- перемещение листа;
- перемещение поддерева;
- удаление узла без поддерева;
- перемещение узла без поддерева;
- получение прямых потомков;
- получение прямого родителя;
- получение всех потомков (с сохранением иерархичности);
- получение всех родителей (с сохранением иерархичности);

In [1]:
import mysql.connector
import pandas as pd
import random 
from faker import Faker
import pandas as pd
from mysql.connector import connect, Error
#import getpass
fake = Faker('ru_RU')
r_host = "localhost"
r_user = "root"
r_password = "aryuna"
r_db = "srs2"
connection = connect(
    host=r_host, 
    user=r_user,
    password=r_password,
    database = r_db,
)
db_cur = connection.cursor()
from queue import Queue

In [2]:
def AddChildren(ancestors):
    generation = []
    while not ancestors.empty():
        descend = random.randint(2, 5) #сколько детей будет у текущего
        new_parent = ancestors.get()
        for i in range(descend): #по очереди создаём
            #print(f"{i} по идее, какой сейчас по счёту ребенок, а {new_parent} - родитель. Детей должно быть {descend}")
            db_cur.execute(f'INSERT INTO breadcrumb (name) VALUES ("{fake.company()}")')
            db_cur.execute(f"""UPDATE breadcrumb SET path = concat(( SELECT path FROM 
    (SELECT path FROM breadcrumb WHERE id = {new_parent}) a), LAST_INSERT_ID(), '/') WHERE id = LAST_INSERT_ID();
            """)
            connection.commit()
        db_cur.execute(f"SELECT id from breadcrumb where path REGEXP '/*{new_parent}/[:digit:]+/$'")
        #if (new_parent == 1):
        #    db_cur.execute(f"SELECT id from breadcrumb where path REGEXP '{new_parent}/[:digit:]+/$'")
        #else:
        #    db_cur.execute(f"SELECT id from breadcrumb where path REGEXP '/{new_parent}/[:digit:]+/$'")
        generation.append(db_cur.fetchall())
    this_gen_par = []
    for _list in generation: #списки развернуть и сделать не список списков, а просто список
        for x in _list:
            this_gen_par.append(int(x[0])) # x[0] чтоб избавиться от кортежа
    #print(this_gen_par) #ПРОБЛЕМА - ПУСТОТА
    return this_gen_par

def NewParents(ancestors, gen):
#сначала достаём всё нынешнее поколение
#потом генерим кол-во детей, которые родят
#а потом рандомсэмплим тех, кто родит и всех их кладём в ancestors
    chil_num = random.randint(1, len(gen)) #сначала генерим то, сколько из детей станет родителями
    gen = random.sample(gen, chil_num) #получаем тех, кто станет родителями
    for x in gen:
        ancestors.put(x) #кладём в очередь

In [3]:
def TreeGenerator(d):
    depth = 0
    ancestors = Queue() #очередь родителей
    #создание корня, id будет 1 из-за автоинкремента
    db_cur.execute(f"INSERT INTO breadcrumb (name, path) VALUES ('{fake.company()}', '1/')")
    ancestors.put(1) #пути предков
    new_generation = AddChildren(ancestors) #получение нынешнего поколения
    #print(ancestors.qsize()) #0 после 1-го раза - потому что в процессе добавления детей всё удалилось
    depth = 1 #Т.к. уже случился AddChildren 
    while (depth < d):
        NewParents(ancestors, new_generation)
        new_generation = AddChildren(ancestors)
        depth += 1
def GetDepth():
    db_cur.execute("""SELECT MAX(LENGTH(path) - LENGTH(replace(path, '/', '')) - 1) from breadcrumb""")
    # макс гл узла с путём в 100 символов 
    # 123456/6789/
    # 1/2/a/v/b/4/
#"SELECT MAX(LENGTH(path) - LENGTH(replace(path, '/', '')) - 1) from breadcrumb"
#"""with t as ( SELECT LENGTH(path) - LENGTH(replace(path, '/', '')) - 1 as delim from breadcrumb )
#SELECT MAX(delim) FROM t"""
    return db_cur.fetchone()[0]
def CheckChildren():
    db_cur.execute("""
    select b1.id, b1.name, count(b2.id) from breadcrumb b1 
JOIN breadcrumb b2 ON b2.path = CONCAT(b1.path, b2.id, '/')
group by b1.id order by b1.id
""")
    #путь потомка равен пути родителя с id потомка и разделителем 
    # равен конкатенации пути и id с разделителем в конце
    #выборка - родители и их потомки (за счёт path = concat). Группируем их по родителям (b2) и считаем потомков каждого
    # условие у b2 - путь равен path+id+/
    for i in db_cur:
        #print(i)
        if i[2] < 2 or i[2] > 5:
            print(f'ЧТО-ТО ПОШЛО НЕ ТАК С ДЕТЬМИ: их {i[2]}, родитель {i[0]}')

In [4]:
def GetParent(v):
    db_cur.execute(f"""SELECT id from breadcrumb WHERE path = 
    (SELECT REPLACE(path, '/{v}/', '/') as t from breadcrumb WHERE path LIKE '%/{v}/')""")
    #db_cur.execute(f"""SELECT id from breadcrumb WHERE path = 
    #(SELECT REGEXP_REPLACE(path, '%/{v}/', '/') from breadcrumb WHERE path LIKE '%/{v}/')""")
    a = db_cur.fetchone()
    if a is not None:
        return a[0]
    else: 
        print(a)
def GetChildren(v):
    v = f"/{v}" if v != 1 else "^1"
    db_cur.execute(f"SELECT id from breadcrumb where path REGEXP '{v}/[:digit:]+/$'")
    a = db_cur.fetchall()
    if a is not None and len(a) > 0:
        children = [x[0] for x in a]
        return children
    else:
        print(f"Детей у вершины нет")
def GetAllChildren(v):
    v = f"/{v}" if v != 1 else "^1"
    db_cur.execute(f"SELECT id from breadcrumb where path REGEXP '{v}/[:digit:]+'")
    for i in db_cur:
        print(i)
def GetAllParents(v): 
    db_cur.execute(f"""WITH RECURSIVE cte AS
(
  SELECT id, name, path from breadcrumb WHERE id = {v}
  UNION ALL
  SELECT b.id, b.name, b.path from breadcrumb b JOIN cte ON b.path =
CONCAT(SUBSTRING_INDEX(cte.path, '/', length(cte.path) - length(replace(cte.path,'/','')) - 1), '/')
)
SELECT * from cte""") #путь родителя должен быть равен пути ребёнка до предпоследнего разделителя
    for i in db_cur:
        print(i)

In [5]:
def AddLeaf(title, parent):
    db_cur.execute(f'INSERT INTO breadcrumb (name) VALUES ("{fake.company()}")')
    db_cur.execute(f"""UPDATE breadcrumb SET path = concat(( SELECT path FROM 
    (SELECT path FROM breadcrumb WHERE id = {parent}) a), LAST_INSERT_ID(), '/') WHERE id = LAST_INSERT_ID();
    """)
    connection.commit()
def DelSubTreeOrLeaf(v):
    #поиск потомков, удаление каждого
    db_cur.execute(f"DELETE from breadcrumb WHERE path like '%/{v}/%';")
    connection.commit()
def MoveSubTreeOrLeaf(v, change):
    #поиск потомков, перемещение каждого
    db_cur.execute(f"""WITH b AS (SELECT path FROM breadcrumb)
    UPDATE breadcrumb SET path = REPLACE(path, (SELECT path FROM b WHERE path like '%/{v}/'), CONCAT(
    (SELECT path from b WHERE path like '%/{change}/'), {v}, '/')
    )""")
    connection.commit()
def DelNodeWithoutTree(node):
    db_cur.execute(f"""UPDATE breadcrumb SET path = REPLACE(path, 
(SELECT path FROM (SELECT path from breadcrumb WHERE path like '%/{node}/') as t), 
(SELECT pp FROM (SELECT REGEXP_REPLACE(path, '/{node}/', '/') as pp from breadcrumb WHERE path like '%/{node}/') tt)
)""")
    #Замена пути на родительский без этой самой вершины
    connection.commit()
    DelSubTreeOrLeaf(node)
def MoveNodeWithoutTree(node, change):
    #перемещаем наверх и перемещаем
    db_cur.execute(f"""UPDATE breadcrumb SET path = REPLACE(path, 
(SELECT path FROM (SELECT path from breadcrumb WHERE path like '%/{node}/') as t), (SELECT pp FROM
(SELECT REGEXP_REPLACE(path, '/{node}/', '/') as pp from breadcrumb WHERE path like '%/{node}/') tt))""")
    connection.commit()
    MoveSubTreeOrLeaf(node, change)

In [11]:
drop = ("DROP TABLE IF EXISTS breadcrumb")
breadcrumb = ("""CREATE TABLE breadcrumb(
 id INT PRIMARY KEY AUTO_INCREMENT,
 name VARCHAR(255),
 path VARCHAR(1000)
)
""") 
db_cur.execute(drop)
db_cur.execute(breadcrumb)
connection.commit()
d = random.randint(10, 15)
TreeGenerator(d)

In [6]:
def Checking():
    db_cur.execute("SELECT COUNT(id) FROM breadcrumb")
    n = db_cur.fetchone()[0]
    print(f"Количество элементов - {n}")
    print(f"Глубина дерева - {GetDepth()}")
    print("Если где-то детей будет не доставать или их будет слишком много, выведется сообщение и программа остановится. В противном случае вас ожидает гордое молчание.")
    CheckChildren()
def Interface():
    tab = ''
    a = input("Введите вершину для махинаций. Если понадобится что-то ещё, введёте потом. ")
    while tab != 'Всё':
        db_cur = connection.cursor()
        db_cur.execute(f"SELECT id FROM breadcrumb where id = {a}")
        check = db_cur.fetchone()
        if check is None:
            a = input("Этой вершины нет, давай по новой: ")
            continue
        tab = input("Введите число от 1 до 9. Или 0: ")
        if tab == '1':
            print(f"Родитель вершины {a}: {GetParent(a)}")
        elif tab == '2':
            print(f"Прямые потомки вершины {a}: {GetChildren(a)}")
        elif tab == '3':
            name = input(f"В вершину {a} добавляется лист. Введите его имя: ")
            AddLeaf(name, a)
        elif tab == '4':
            print(f'Удаляется вершина {a} (с поддеревом, если оно есть). ')
            DelSubTreeOrLeaf(a)
        elif tab == '5':
            new = input(f"Куда переместить вершину {a} (с поддеревом, если оно есть)? ")
            MoveSubTreeOrLeaf(a, new)
        elif tab == '6':
            print(f'Удаляется вершина {a} (без поддерева). ')
            DelNodeWithoutTree(a)
        elif tab == '7':
            new = input(f"Куда переместить вершину {a} (без поддерева)? ")
            MoveNodeWithoutTree(a, new)
        elif tab == '8':
            print(f"Получение ВСЕХ РОДИТЕЛЕЙ до {a}!!! ")
            GetAllParents(a)
        elif tab == '9':
            print(f"Получение ВСЕХ ПОТОМКОВ {a}!!! ")
            GetAllChildren(a)
        elif tab == '0':
            a = input("Новая вершина для махинаций: ")
        elif tab == 'Проверка':
            Checking()
        elif tab == "Всё" or tab == ".":
            print("Oк. ")
        else:
            print("Вы ввели что-то не то... ")
    depth = 0
    print("До свидания.")

# ы
    1 - прямой родитель
    2 - прямые потомки
    3 - добавление листа
    4 - удаление листа/поддерева
    5 - перемещение листа/поддерева
    6 - удаление узла без поддерева
    7 - перемещение узла без поддерева
    8 - получение всех потомков (с сохранением иерархичности)
    9 - получение всех родителей (с сохранением иерархичности)
    0 - выбор другой вершины

In [8]:
Interface()

Введите вершину для махинаций. Если понадобится что-то ещё, введёте потом. 99
Введите число от 1 до 9. Или 0: Проверка
Количество элементов - 742
Глубина дерева - 10
Если где-то детей будет не доставать или их будет слишком много, выведется сообщение и программа остановится. В противном случае вас ожидает гордое молчание.
Введите число от 1 до 9. Или 0: Всё
Oк. 
До свидания.


In [7]:
Checking()
#db_cur.execute(f"""SELECT id from breadcrumb where path REGEXP '1/[:digit:]+/$'""")
#print(db_cur.fetchall() is not None)
#GetParent(1)

Количество элементов - 742
Глубина дерева - 10
Если где-то детей будет не доставать или их будет слишком много, выведется сообщение и программа остановится. В противном случае вас ожидает гордое молчание.


db_cur.execute(f"""UPDATE breadcrumb SET path = concat(( SELECT path
FROM (SELECT path
FROM breadcrumb WHERE id = LAST_INSERT_ID()) a), LAST_INSERT_ID(), '/') WHERE id =
LAST_INSERT_ID();
""") #сработает только для детей корня и текущих потомков 1 ветки